<a href="https://colab.research.google.com/github/hiteshkukreja100/Call_Quality_Analyser/blob/main/Call_Quality_Analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download Audio from Youtube

In [1]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 934.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 13.4 MB/s eta 0:00:00


In [2]:
import yt_dlp
import os

def download_audio_from_youtube(url, output_dir="downloads"):
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, "%(id)s.%(ext)s")

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_path,
        'quiet': True,
        'postprocessors': [
            {'key': 'FFmpegExtractAudio', 'preferredcodec': 'wav', 'preferredquality': '192'},
        ],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        audio_path = os.path.join(output_dir, f"{info['id']}.wav")

    return audio_path

In [3]:
youtube_url = "https://www.youtube.com/watch?v=4ostqJD3Psc"  # enter your youtube url

raw_audio = download_audio_from_youtube(youtube_url)
print(f"Raw audio saved at: {raw_audio}")

Raw audio saved at: downloads/4ostqJD3Psc.wav
